## PyTorch 实现变分自编码器 VAE
理论部分参考：[自编码器变形和变分自编码器理论介绍及其 PyTorch 实现](https://dreamhomes.github.io/posts/202006021200.html)

In [1]:
import os

import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.utils import save_image

print(torch.__version__)

1.5.0


In [2]:
# 参数设置

# Create a directory if not exists
output_dir = 'output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
image_size = 28*28
h_dim = 400
z_dim = 20
num_epochs = 20
batch_size = 64
learning_rate = 1e-3

## Mnist dataset

In [3]:
# Mnist dataset
dataset = torchvision.datasets.MNIST(root='./data/mnist/',
                                     train=True,
                                     transform=torchvision.transforms.ToTensor(),
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True,
                                          num_workers=0)

## Model

In [4]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    # 重参数
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

model = VAE().to(device)

### Train

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and kl divergence
        # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 200 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(output_dir, 'sampled-{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(output_dir, 'reconst-{}.png'.format(epoch+1)))

D:\DevTools\Miniconda3\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\DevTools\Miniconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/20], Step [200/938], Reconst Loss: 8821.7510, KL Div: 891.5093
Epoch[1/20], Step [400/938], Reconst Loss: 7804.7573, KL Div: 1056.6965
Epoch[1/20], Step [600/938], Reconst Loss: 6771.7695, KL Div: 1316.7372
Epoch[1/20], Step [800/938], Reconst Loss: 6485.7046, KL Div: 1353.1658
Epoch[2/20], Step [200/938], Reconst Loss: 6101.1880, KL Div: 1499.0868
Epoch[2/20], Step [400/938], Reconst Loss: 6246.6777, KL Div: 1568.8821
Epoch[2/20], Step [600/938], Reconst Loss: 5953.7402, KL Div: 1547.8967
Epoch[2/20], Step [800/938], Reconst Loss: 5708.8115, KL Div: 1573.6791
Epoch[3/20], Step [200/938], Reconst Loss: 5529.4912, KL Div: 1565.7269
Epoch[3/20], Step [400/938], Reconst Loss: 5354.6104, KL Div: 1531.8453
Epoch[3/20], Step [600/938], Reconst Loss: 5838.3325, KL Div: 1591.4172
Epoch[3/20], Step [800/938], Reconst Loss: 5333.0142, KL Div: 1538.3901
Epoch[4/20], Step [200/938], Reconst Loss: 5248.3931, KL Div: 1564.2769
Epoch[4/20], Step [400/938], Reconst Loss: 5570.7324, KL Div: 158